# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

from keras import optimizers, Sequential
from keras import regularizers
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1409962315762049025
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 8624587596754019954
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French data/set:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French data/set:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    text_tokenizer = Tokenizer()
    text_tokenizer.fit_on_texts(x)
    text_tokenized = text_tokenizer.texts_to_sequences(x)
    return text_tokenized, text_tokenizer
tests.test_tokenize(tokenize)

text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length == None:
        length = max([len(sentance) for sentance in x])
    x = np.array(x)
    x_padded = pad_sequences(x, maxlen=length, dtype='int32', padding='post')
    return x_padded

tests.test_pad(pad)

test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [24]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    number_units = 100
    model = Sequential()
    model.add(GRU(number_units, return_sequences=True, input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size)))
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(lr=0.0014, decay=0.00004),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

tmp_x = pad(preproc_english_sentences, 21)
tmp_x = tmp_x.reshape((-1, 21, 1))

model = simple_model(tmp_x.shape,
                     max_french_sequence_length,
                     english_vocab_size,
                     french_vocab_size)

model.fit(tmp_x, 
          preproc_french_sentences, 
          batch_size=500, 
          epochs=20, 
          validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_acc', patience=3)])

print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 10s 91us/step - loss: 5.8705 - acc: 0.4424 - val_loss: nan - val_acc: 0.4627
Epoch 2/20
110288/110288 [==============================] - 9s 78us/step - loss: 5.8631 - acc: 0.4077 - val_loss: nan - val_acc: 0.4024
Epoch 3/20
110288/110288 [==============================] - 9s 78us/step - loss: 5.5271 - acc: 0.4088 - val_loss: nan - val_acc: 0.3851
Epoch 4/20
110288/110288 [==============================] - 9s 78us/step - loss: 4.6631 - acc: 0.4082 - val_loss: nan - val_acc: 0.4199
à à est à à au au au à <PAD> à au au <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [27]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    word_vec_dim = 20
    number_units = 30
    inputs = Input(shape=input_shape[1:])
    embedding = Embedding(input_dim=english_vocab_size, output_dim=word_vec_dim)(inputs)
    hidden = GRU(number_units, return_sequences=True)(embedding)
    outputs = TimeDistributed(Dense(french_vocab_size))(hidden)
    model = Model(inputs, Activation('softmax')(outputs))
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(lr=0.0016, decay=0.00003),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)

tmp_x = pad(preproc_english_sentences, 21)
model = embed_model(tmp_x.shape,
                    max_french_sequence_length,
                    english_vocab_size,
                    french_vocab_size)

model.fit(tmp_x, 
          preproc_french_sentences, 
          batch_size=500, 
          epochs=20, 
          validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_acc', patience=2)])

print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 12s 113us/step - loss: 3.4753 - acc: 0.3982 - val_loss: nan - val_acc: 0.4179
Epoch 2/20
110288/110288 [==============================] - 10s 94us/step - loss: 2.2740 - acc: 0.5070 - val_loss: nan - val_acc: 0.5785
Epoch 3/20
110288/110288 [==============================] - 10s 95us/step - loss: 1.4924 - acc: 0.6290 - val_loss: nan - val_acc: 0.6813
Epoch 4/20
110288/110288 [==============================] - 10s 94us/step - loss: 1.0993 - acc: 0.7230 - val_loss: nan - val_acc: 0.7549
Epoch 5/20
110288/110288 [==============================] - 10s 95us/step - loss: 0.8896 - acc: 0.7714 - val_loss: nan - val_acc: 0.7884
Epoch 6/20
110288/110288 [==============================] - 10s 95us/step - loss: 0.7651 - acc: 0.7979 - val_loss: nan - val_acc: 0.8083
Epoch 7/20
110288/110288 [==============================] - 11s 95us/step - loss: 0.6873 - acc: 0.8149 - val_loss: nan - val_a

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [25]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # This model and the first one did not produce any good results.
    # I tried many different GRU untis but the fact that the 
    # data is represented by continuous data made it too difficult.
    number_units = 100
    model = Sequential()
    model.add(Bidirectional(GRU(number_units, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size)))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(lr=0.0014, decay=0.00003),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)

tmp_x = pad(preproc_english_sentences, 21)
tmp_x = tmp_x.reshape((-1, 21, 1))
model = bd_model(tmp_x.shape,
                 max_french_sequence_length,
                 english_vocab_size,
                 french_vocab_size)

model.fit(tmp_x, 
          preproc_french_sentences, 
          batch_size=500, 
          epochs=20, 
          validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_acc', patience=2)])

print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 17s 154us/step - loss: 6.7311 - acc: 0.3857 - val_loss: nan - val_acc: 0.2545
Epoch 2/20
110288/110288 [==============================] - 15s 138us/step - loss: 6.5827 - acc: 0.3850 - val_loss: nan - val_acc: 0.4057
Epoch 3/20
110288/110288 [==============================] - 15s 136us/step - loss: 6.3285 - acc: 0.4084 - val_loss: nan - val_acc: 0.4194
Epoch 4/20
110288/110288 [==============================] - 15s 137us/step - loss: 6.2747 - acc: 0.4208 - val_loss: nan - val_acc: 0.4223
Epoch 5/20
110288/110288 [==============================] - 15s 136us/step - loss: 6.1729 - acc: 0.4184 - val_loss: nan - val_acc: 0.4168
Epoch 6/20
110288/110288 [==============================] - 15s 136us/step - loss: 5.8612 - acc: 0.3949 - val_loss: nan - val_acc: 0.4507
Epoch 7/20
110288/110288 [==============================] - 15s 136us/step - loss: 5.0917 - acc: 0.4319 - val_loss: nan -

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.
## Prediction (IMPLEMENTATION)

In [65]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    word_vec_dim = 20
    number_units = 30
    inputs = Input(shape=input_shape[1:])
    embedding = Embedding(input_dim=english_vocab_size, output_dim=word_vec_dim)(inputs)
    hidden1 = Bidirectional(GRU(number_units, return_sequences=True))(embedding)
    hidden2 = Bidirectional(GRU(number_units, return_sequences=True))(hidden1)
    output = TimeDistributed(Dense(french_vocab_size))(hidden2)
    
    model = Model(inputs, Activation('softmax')(output))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.0016, decay=0.00001),
                  metrics=['accuracy'])
    return model

# tests.test_model_final(model_final) <- My model is not a Encoder-Decoder and therefore the input and output shape
# are the same.

print('Final Model Loaded')

def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    
    tmp_x = pad(x, 21)

    model = model_final(tmp_x.shape,
                        max_french_sequence_length,
                        english_vocab_size,
                        french_vocab_size)

    model.fit(tmp_x, 
              preproc_french_sentences, 
              batch_size=500, 
              epochs=100, 
              validation_split=0.2,
              callbacks=[EarlyStopping(monitor='val_acc', patience=4)])
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    # My model is not an Encoder-Decoder and therefore the input must be reshaped.
    sentences = pad(sentences, 21)
    # 
    predictions = model.predict(sentences, len(sentences))
        
    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/100
110288/110288 [==============================] - 40s 366us/step - loss: 2.8156 - acc: 0.4629 - val_loss: nan - val_acc: 0.5336
Epoch 2/100
110288/110288 [==============================] - 31s 282us/step - loss: 1.5458 - acc: 0.6185 - val_loss: nan - val_acc: 0.6866
Epoch 3/100
110288/110288 [==============================] - 32s 286us/step - loss: 1.0637 - acc: 0.7270 - val_loss: nan - val_acc: 0.7615
Epoch 4/100
110288/110288 [==============================] - 31s 285us/step - loss: 0.8212 - acc: 0.7868 - val_loss: nan - val_acc: 0.8095
Epoch 5/100
110288/110288 [==============================] - 31s 280us/step - loss: 0.6650 - acc: 0.8234 - val_loss: nan - val_acc: 0.8351
Epoch 6/100
110288/110288 [==============================] - 31s 283us/step - loss: 0.5620 - acc: 0.8454 - val_loss: nan - val_acc: 0.8549
Epoch 7/100
110288/110288 [==============================] - 31s 280us/step - loss: 0.4918 - acc

110288/110288 [==============================] - 31s 284us/step - loss: 0.1107 - acc: 0.9669 - val_loss: nan - val_acc: 0.9611
Epoch 60/100
110288/110288 [==============================] - 31s 283us/step - loss: 0.1106 - acc: 0.9668 - val_loss: nan - val_acc: 0.9625
Epoch 61/100
110288/110288 [==============================] - 31s 282us/step - loss: 0.1097 - acc: 0.9671 - val_loss: nan - val_acc: 0.9638
Epoch 62/100
110288/110288 [==============================] - 31s 283us/step - loss: 0.1075 - acc: 0.9679 - val_loss: nan - val_acc: 0.9640
Epoch 63/100
110288/110288 [==============================] - 31s 284us/step - loss: 0.1067 - acc: 0.9681 - val_loss: nan - val_acc: 0.9630
Epoch 64/100
110288/110288 [==============================] - 31s 281us/step - loss: 0.1065 - acc: 0.9681 - val_loss: nan - val_acc: 0.9649
Epoch 65/100
110288/110288 [==============================] - 31s 282us/step - loss: 0.1054 - acc: 0.9685 - val_loss: nan - val_acc: 0.9651
Epoch 66/100
110288/110288 [=====

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [67]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 344689 bytes to machine_translation.html']

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    return None
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)